<a href="https://colab.research.google.com/github/fabiohsst/LinkPredictionGNN/blob/main/link_prediction_heterogeneous_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author:** Fabio Tavares

**Email:** fabio.tavares.fma@gmail.com

**LinkedIn:** [Fabiohsst](https://www.linkedin.com/in/fabiohsst/)

# Importing required packages

In [ ]:
import torch
from torch import Tensor
print(torch.__version__)

2.0.1+cu118


In [ ]:
# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 41.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.1 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/nightly/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.0 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-55ysinzj
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-55ysinzj
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 143487bc18a5bf3e7968c2637823791781e6135a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl 

## Heterogeneous Graph Creation


In [ ]:
# Import datasets
import pandas as pd

# Defines columns name
column_names = ["userid", "timestamp", "artistid", "artist-name", "trackid", "track-name"]

# Load songs dataset
songs = pd.read_csv('/content/drive/MyDrive/Thesis/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
                    sep='\t',names=column_names, error_bad_lines=False)

# Load users dataset
users = pd.read_csv('/content/drive/MyDrive/Thesis/lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False)


<ipython-input-5-0a89e584a9d5>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  songs = pd.read_csv('/content/drive/MyDrive/Thesis/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
Skipping line 2120260: expected 6 fields, saw 8

Skipping line 2446318: expected 6 fields, saw 8

Skipping line 11141081: expected 6 fields, saw 8

Skipping line 11152099: expected 6 fields, saw 12
Skipping line 11152402: expected 6 fields, saw 8

Skipping line 11882087: expected 6 fields, saw 8

Skipping line 12902539: expected 6 fields, saw 8
Skipping line 12935044: expected 6 fields, saw 8

Skipping line 17589539: expected 6 fields, saw 8

<ipython-input-5-0a89e584a9d5>:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('/content/drive/MyDrive/Thesis/lastfm-dataset-1K/userid-profil

In [ ]:
# Print songs and users dataset
print('songs:')
print('===========')
print(songs.head())
print()
print('users:')
print('============')
print(users.head())

songs:
        userid             timestamp                              artistid  \
0  user_000001  2009-05-04T23:08:57Z  f1b1cf71-bd35-4e99-8624-24a6e15f133a   
1  user_000001  2009-05-04T13:54:10Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
2  user_000001  2009-05-04T13:52:04Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
3  user_000001  2009-05-04T13:42:52Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
4  user_000001  2009-05-04T13:42:11Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   

  artist-name trackid                                  track-name  
0   Deep Dish     NaN  Fuck Me Im Famous (Pacha Ibiza)-09-28-2007  
1        坂本龍一     NaN           Composition 0919 (Live_2009_4_15)  
2        坂本龍一     NaN                        Mc2 (Live_2009_4_15)  
3        坂本龍一     NaN                     Hibari (Live_2009_4_15)  
4        坂本龍一     NaN                        Mc1 (Live_2009_4_15)  

users:
           #id gender   age        country    registered
0  user_000001      m   NaN          Japan  Aug 13,

## Data cleaning and Feature engineering

In [ ]:
# Remove missing values on 'songs' dataset
songs.dropna(inplace=True)

In [ ]:
# Handle missing values on users dataset

# For numerical columns
mean_age = users['age'].mean()
mean_age_rounded = round(mean_age)
users['age'].fillna(mean_age_rounded, inplace=True)
users['age'] = users['age'].astype(int)

# Missing gender
users['gender'].fillna('Unknown', inplace=True)

# Missing Country
users['country'].fillna('Unknown', inplace=True)

# Drop remaining missing values
users.dropna(inplace=True)

In [ ]:
users.dtypes

#id           object
gender        object
age            int64
country       object
registered    object
dtype: object

In [ ]:
# Double check missing values
missing_values_songs = songs.isnull().sum()
missing_values_users = users.isnull().sum()

print('songs missings values:')
print(missing_values_songs)
print('======================')
print('users missings values:')
print(missing_values_users)

songs missings values:
userid         0
timestamp      0
artistid       0
artist-name    0
trackid        0
track-name     0
dtype: int64
users missings values:
#id           0
gender        0
age           0
country       0
registered    0
dtype: int64


### Sampling dataset

In [ ]:
# 1. Sample users
sampled_users = users.sample(frac=0.2, random_state=42)

# 2. Use the sampled users to filter the songs dataset.
sampled_songs_data = songs[songs['userid'].isin(sampled_users['#id'])]

# 3. Sample the songs dataset
sampled_songs_data = sampled_songs_data.sample(frac=0.1, random_state=42)

# 4. Filter the users dataset to include only users present in the sampled_songs_data
final_users = sampled_users[sampled_users['#id'].isin(sampled_songs_data['userid'])]

### Creates new feature for tracks

In [ ]:
# Convert the 'timestamp' column to datetime format
sampled_songs_data['timestamp'] = pd.to_datetime(sampled_songs_data['timestamp'])

# Convert the 'registered' column to datetime format and coerce any errors
final_users['registered'] = pd.to_datetime(users['registered'], errors='coerce')

# Assuming first timestamp as "release date"
song_release_date = sampled_songs_data.groupby('track-name')['timestamp'].min()
sampled_songs_data['song_release_date'] = sampled_songs_data['track-name'].map(song_release_date)
sampled_songs_data['days_since_release'] = (sampled_songs_data['timestamp'] - sampled_songs_data['song_release_date']).dt.days


<ipython-input-12-a8b139a86963>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_users['registered'] = pd.to_datetime(users['registered'], errors='coerce')


In [ ]:
# print datasets shape
print(f'Users sampled shape:{final_users.shape}')
print(f'Songs sampled shape:{sampled_songs_data.shape}')

Users sampled shape:(196, 5)
Songs sampled shape:(340364, 8)


In [ ]:
# check songs columns
sampled_songs_data.columns

Index(['userid', 'timestamp', 'artistid', 'artist-name', 'trackid',
       'track-name', 'song_release_date', 'days_since_release'],
      dtype='object')

In [ ]:
# List of unique countries, genders and age
unique_countries = final_users['country'].unique()
unique_genders = final_users['gender'].unique()
age = final_users['age']

print(f'Countries on the dataset:{unique_countries} a total of {unique_countries.size} countries')
print('==================================================')
print(f'Genders values on the dataset:{unique_genders}')

Countries on the dataset:['Argentina' 'Canada' 'Unknown' 'Finland' 'United States' 'Netherlands'
 'Norway' 'Sweden' 'Tunisia' 'Macedonia' 'Bulgaria' 'Czech Republic'
 'Latvia' 'Australia' 'Japan' 'Germany' 'United Kingdom' 'Algeria'
 'Poland' 'Turkey' 'Italy' 'Peru' 'Portugal' 'Brazil' 'Mexico'
 'Antarctica' 'Estonia' 'Armenia' 'Belgium' 'Chile' 'New Zealand'
 'Russian Federation' 'Lithuania' 'France' 'Greece' 'Trinidad and Tobago'
 'Serbia' 'Spain' 'Slovakia' "Korea, Democratic People's Republic of"
 'Romania' 'Slovenia' 'Colombia'] a total of 43 countries
Genders values on the dataset:['m' 'f' 'Unknown']


In [ ]:
# convert columns to dummies values
country_dummies = final_users['country'].str.get_dummies()
gender_dummies = final_users['gender'].str.get_dummies()
print(country_dummies[['Argentina', 'Canada', 'Unknown', 'Finland']].head())
print()
print('=======================================')
print(gender_dummies.head())


# Combine dummies and age columns
combined_dummies = pd.concat([country_dummies, gender_dummies,age], axis=1)

# User features
users_feat = torch.from_numpy(combined_dummies.values).to(torch.float)
assert users_feat.size() ==(196, 47)

     Argentina  Canada  Unknown  Finland
618          1       0        0        0
455          0       1        0        0
737          0       0        1        0
440          0       0        0        1
278          0       0        0        0

     Unknown  f  m
618        0  0  1
455        0  1  0
737        0  0  1
440        1  0  0
278        0  1  0


In [ ]:
# Track feature
days_since_release = sampled_songs_data['days_since_release']
track_feat = torch.from_numpy(days_since_release.values).to(torch.float)
assert track_feat.size() ==(340364,)

The `ratings.csv` data connects users (as given by `userId`) and movies (as given by `movieId`).
Due to simplicity, we do not make use of the additional `timestamp` and `rating` information.
Here, we first read the `*.csv` file from disk, and create a mapping that maps entry IDs to a consecutive value in the range `{ 0, ..., num_rows - 1 }`.
This is needed as we want our final data representation to be as compact as possible, *e.g.*, the representation of a movie in the first row should be accessible via `x[0]`.

Afterwards, we obtain the final `edge_index` representation of shape `[2, num_ratings]` from `ratings.csv` by merging mapped user and movie indices with the raw indices given by the original data frame.

### Mapping unique IDs

In [ ]:
# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = sampled_songs_data['userid'].unique()
unique_user_id = pd.DataFrame(data={
    'userid': unique_user_id,
    'mappedID': pd.RangeIndex(len(unique_user_id)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_id.head())
print()

# Create a mapping from unique track indices to range [0, num_track_nodes):
unique_track_id = sampled_songs_data['trackid'].unique()
unique_track_id = pd.DataFrame(data={
    'trackid': unique_track_id,
    'mappedID': pd.RangeIndex(len(unique_track_id)),
})
print("Mapping of track IDs to consecutive values:")
print("===========================================")
print(unique_track_id.head())

# Create a mapping from unique artist indices to range [0, num_artist_nodes):
unique_artist_id = sampled_songs_data['artistid'].unique()
unique_artist_id = pd.DataFrame(data={
    'artistid': unique_artist_id,
    'mappedID': pd.RangeIndex(len(unique_artist_id)),
})
print("Mapping of artist IDs to consecutive values:")
print("===========================================")
print(unique_artist_id.head())

# Perform merge to obtain the edges from users and tracks:
user_id = pd.merge(sampled_songs_data['userid'], unique_user_id,
                            left_on='userid', right_on='userid', how='left')
user_id = torch.from_numpy(user_id['mappedID'].values)
track_id = pd.merge(sampled_songs_data['trackid'], unique_track_id,
                            left_on='trackid', right_on='trackid', how='left')
track_id = torch.from_numpy(track_id['mappedID'].values)
artist_id = pd.merge(sampled_songs_data['artistid'], unique_artist_id,
                            left_on='artistid', right_on='artistid', how='left')
artist_id = torch.from_numpy(artist_id['mappedID'].values)

# construct `edge_index` in COO format
# following PyG semantics:
edge_index_user_to_track = torch.stack([user_id, track_id], dim=0)
assert edge_index_user_to_track.size() == (2, 340364)
edge_index_user_to_artist = torch.stack([user_id, artist_id], dim=0)
assert edge_index_user_to_artist.size() == (2, 340364)

print()
print("Final edge indices pointing from users to tracks:")
print("=================================================")
print(edge_index_user_to_track)
print()
print("Final edge indices pointing from users to artists:")
print("=================================================")
print(edge_index_user_to_track)

Mapping of user IDs to consecutive values:
        userid  mappedID
0  user_000188         0
1  user_000142         1
2  user_000748         2
3  user_000329         3
4  user_000792         4

Mapping of track IDs to consecutive values:
                                trackid  mappedID
0  423672d5-14e9-4068-b198-8e5724e7342c         0
1  618566d5-2ff5-4388-8947-03ceba773483         1
2  82e9a44d-6c71-4bcd-9719-7dfdfe8ede41         2
3  24436eea-e79f-4592-b7f8-3e9bfe01f483         3
4  f073f081-d9ad-4b1f-9789-070d07481eb5         4
Mapping of artist IDs to consecutive values:
                               artistid  mappedID
0  5e521e8c-0ab2-44c4-8fd8-14d8d3321265         0
1  c707b37e-cb5c-4694-be4d-46beefcb2475         1
2  3525b27c-e5a5-4d1c-b80b-f223696bf675         2
3  c485632c-b784-4ee9-8ea1-c5fb365681fc         3
4  7d31729b-4a6e-422a-9919-eb225a511fb0         4

Final edge indices pointing from users to tracks:
tensor([[     0,      1,      2,  ...,     36,     22,     90],
  

## Link Prediction: Users and tracks

### Create heterogeneous graph with HeteroData - Users and Tracks interaction

In [ ]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data_track = HeteroData()

# Save node indices:
data_track["user"].node_id = torch.arange(len(unique_user_id))
data_track["track"].node_id = torch.arange(len(sampled_songs_data))

# Add the node features and edge indices:
data_track["user"].x = users_feat
data_track["track"].x = track_feat
data_track["user", "listened_to", "track"].edge_index = edge_index_user_to_track

# Add reverse edges using `T.ToUndirected()`
data_track = T.ToUndirected()(data_track)

print(data_track)

assert data_track.node_types == ["user", "track"]
assert data_track.edge_types == [("user", "listened_to", "track"),
                           ("track", "rev_listened_to", "user")]
assert data_track["user"].num_nodes == 196
assert data_track["user"].num_features == 47
assert data_track["track"].num_nodes == 340364
assert data_track["track"].num_features == 1
assert data_track["user", "listened_to", "track"].num_edges == 340364
assert data_track["track", "rev_listened_to", "user"].num_edges == 340364

HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  track={
    node_id=[340364],
    x=[340364],
  },
  (user, listened_to, track)={ edge_index=[2, 340364] },
  (track, rev_listened_to, user)={ edge_index=[2, 340364] }
)


### Defining Edge-level Training Splits


In [ ]:
# Split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, 70% of edges is used for message passing,
# and 30% of edges for supervision.
# Generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.

# Leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("user", "listened_to", "track"),
    rev_edge_types=("track", "rev_listened_to", "user")
)

train_data, val_data, test_data = transform(data_track)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)

assert train_data["user", "listened_to", "track"].num_edges == 190605
assert train_data["user", "listened_to", "track"].edge_label_index.size(1) == 81687
assert train_data["track", "rev_listened_to", "user"].num_edges == 190605
# No negative edges added:
assert train_data["user", "listened_to", "track"].edge_label.min() == 1
assert train_data["user", "listened_to", "track"].edge_label.max() == 1

assert val_data["user", "listened_to", "track"].num_edges == 272292
assert val_data["user", "listened_to", "track"].edge_label_index.size(1) == 102108
assert val_data["track", "rev_listened_to", "user"].num_edges == 272292
# Negative edges with ratio 2:1:
assert val_data["user", "listened_to", "track"].edge_label.long().bincount().tolist() == [68072, 34036]

Training data:
HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  track={
    node_id=[340364],
    x=[340364],
  },
  (user, listened_to, track)={
    edge_index=[2, 190605],
    edge_label=[81687],
    edge_label_index=[2, 81687],
  },
  (track, rev_listened_to, user)={ edge_index=[2, 190605] }
)

Validation data:
HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  track={
    node_id=[340364],
    x=[340364],
  },
  (user, listened_to, track)={
    edge_index=[2, 272292],
    edge_label=[102108],
    edge_label_index=[2, 102108],
  },
  (track, rev_listened_to, user)={ edge_index=[2, 272292] }
)


### Defining Mini-batch Loaders

In [ ]:
# In the first hop, we sample at most 20 neighbors.
# firs hop: 20 neighbours
# In the second hop, we sample at most 10 neighbors.
# second hop: 10 neighbours
# During training, negative edges are sampled on-the-fly with a ratio of 2:1.

from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["user", "listened_to", "track"].edge_label_index
edge_label = train_data["user", "listened_to", "track"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "listened_to", "track"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "listened_to", "track"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "listened_to", "track"].edge_label.min() == 0
assert sampled_data["user", "listened_to", "track"].edge_label.max() == 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[194],
    x=[194, 47],
    n_id=[194],
    num_sampled_nodes=[3],
  },
  track={
    node_id=[3644],
    x=[3644],
    n_id=[3644],
    num_sampled_nodes=[3],
  },
  (user, listened_to, track)={
    edge_index=[2, 11563],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[11563],
    num_sampled_edges=[2],
    input_id=[128],
  },
  (track, rev_listened_to, user)={
    edge_index=[2, 3387],
    e_id=[3387],
    num_sampled_edges=[2],
  }
)


### Creating Heterogeneous Link-level GNN


In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_track: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_track = x_track[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_track).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.track_lin = torch.nn.Linear(1, hidden_channels)
        self.user_lin = torch.nn.Linear(47, hidden_channels)
        self.track_emb = torch.nn.Embedding(data_track["track"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data_track.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_lin(data["user"].x),
          "track": self.track_lin(data["track"].x.view(-1, 1)) + self.track_emb(data["track"].node_id),
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["track"],
            data["user", "listened_to", "track"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=64)

print(model)

Model(
  (track_lin): Linear(in_features=1, out_features=64, bias=True)
  (user_lin): Linear(in_features=47, out_features=64, bias=True)
  (track_emb): Embedding(340364, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__listened_to__track): SAGEConv(64, 64, aggr=mean)
      (track__rev_listened_to__user): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__listened_to__track): SAGEConv(64, 64, aggr=mean)
      (track__rev_listened_to__user): SAGEConv(64, 64, aggr=mean)
    )
  )
  (classifier): Classifier()
)


### Training Heterogeneous Link-level GNN

In [ ]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Delete later
print("Shape of track.x:", data_track["track"].x.shape)
print("Shape of track.node_id:", data_track["track"].node_id.shape)


for epoch in range(1, 6):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["user", "listened_to", "track"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'
Shape of track.x: torch.Size([340364])
Shape of track.node_id: torch.Size([340364])


100%|██████████| 639/639 [00:07<00:00, 85.18it/s]


Epoch: 001, Loss: 235.4205


100%|██████████| 639/639 [00:07<00:00, 86.01it/s]


Epoch: 002, Loss: 2.1514


...

Epoch: 003, Loss: 0.9699


100%|██████████| 639/639 [00:07<00:00, 85.76it/s]


Epoch: 004, Loss: 0.6433


100%|██████████| 639/639 [00:07<00:00, 87.49it/s]

Epoch: 005, Loss: 0.5455


### Evaluating Heterogeneous Link-level GNN


In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "listened_to", "track"].edge_label_index
edge_label = val_data["user", "listened_to", "track"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "listened_to", "track"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "listened_to", "track"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "listened_to", "track"].edge_label.min() >= 0
assert sampled_data["user", "listened_to", "track"].edge_label.max() <= 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[183],
    x=[183, 47],
    n_id=[183],
    num_sampled_nodes=[3],
  },
  track={
    node_id=[3140],
    x=[3140],
    n_id=[3140],
    num_sampled_nodes=[3],
  },
  (user, listened_to, track)={
    edge_index=[2, 11847],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[11847],
    num_sampled_edges=[2],
    input_id=[384],
  },
  (track, rev_listened_to, user)={
    edge_index=[2, 2866],
    e_id=[2866],
    num_sampled_edges=[2],
  }
)


In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data["user", "listened_to", "track"].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Calculate ROC-AUC
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

# Calculate Average Precision (AP)
ap = average_precision_score(ground_truth, pred)
print(f"Average Precision (AP): {ap:.4f}")

# Calculate F1 score
# First, we need to convert the probabilities to class labels (0 or 1)
# We use 0.5 as the threshold
pred_labels = (pred > 0.5).astype(int)
f1 = f1_score(ground_truth, pred_labels)
print(f"F1 Score: {f1:.4f}")

100%|██████████| 266/266 [00:05<00:00, 50.94it/s] 



Validation AUC: 0.7703
Average Precision (AP): 0.6168
F1 Score: 0.6258


## Link prediction: Users and Artists

### Create heterogeneous graph with HeteroData - Users and artists interaction

In [ ]:
# Define graph to user and artist interaction
data_artist = HeteroData()

# Save node indices:
data_artist["user"].node_id = torch.arange(len(unique_user_id))
data_artist["artist"].node_id = torch.arange(len(sampled_songs_data))

# Add the node features and edge indices:
data_artist["user"].x = users_feat
data_artist["user", "listened_to_artist", "artist"].edge_index = edge_index_user_to_artist

# Add reverse edges using `T.ToUndirected()`
data_artist = T.ToUndirected()(data_artist)

print(data_artist)

assert data_artist.node_types == ["user","artist"]
assert data_artist.edge_types == [("user", "listened_to_artist", "artist"),
                           ("artist", "rev_listened_to_artist", "user")]
assert data_artist["user"].num_nodes == 196
assert data_artist["user"].num_features == 47
assert data_artist["artist"].num_nodes == 340364
assert data_artist["artist"].num_features == 0
assert data_artist["user", "listened_to_artist", "artist"].num_edges == 340364
assert data_artist["artist", "rev_listened_to_artist", "user"].num_edges == 340364

HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  artist={ node_id=[340364] },
  (user, listened_to_artist, artist)={ edge_index=[2, 340364] },
  (artist, rev_listened_to_artist, user)={ edge_index=[2, 340364] }
)


### Defining Edge-level Training Splits


In [ ]:
# Split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, 70% of edges is used for message passing,
# and 30% of edges for supervision.
# Generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.

# Leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("user", "listened_to_artist", "artist"),
    rev_edge_types=("artist", "rev_listened_to_artist", "user")
)

train_data, val_data, test_data = transform(data_artist)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)

assert train_data["user", "listened_to_artist", "artist"].num_edges == 190605
assert train_data["user", "listened_to_artist", "artist"].edge_label_index.size(1) == 81687
assert train_data["artist", "rev_listened_to_artist", "user"].num_edges == 190605
# No negative edges added:
assert train_data["user", "listened_to_artist", "artist"].edge_label.min() == 1
assert train_data["user", "listened_to_artist", "artist"].edge_label.max() == 1

assert val_data["user", "listened_to_artist", "artist"].num_edges == 272292
assert val_data["user", "listened_to_artist", "artist"].edge_label_index.size(1) == 102108
assert val_data["artist", "rev_listened_to_artist", "user"].num_edges == 272292
# Negative edges with ratio 2:1:
assert val_data["user", "listened_to_artist", "artist"].edge_label.long().bincount().tolist() == [68072, 34036]

Training data:
HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  artist={ node_id=[340364] },
  (user, listened_to_artist, artist)={
    edge_index=[2, 190605],
    edge_label=[81687],
    edge_label_index=[2, 81687],
  },
  (artist, rev_listened_to_artist, user)={ edge_index=[2, 190605] }
)

Validation data:
HeteroData(
  user={
    node_id=[196],
    x=[196, 47],
  },
  artist={ node_id=[340364] },
  (user, listened_to_artist, artist)={
    edge_index=[2, 272292],
    edge_label=[102108],
    edge_label_index=[2, 102108],
  },
  (artist, rev_listened_to_artist, user)={ edge_index=[2, 272292] }
)


### Defining Mini-batch Loaders

In [ ]:
# In the first hop, we sample at most 20 neighbors.
# firs hop: 20 neighbours
# In the second hop, we sample at most 10 neighbors.
# second hop: 10 neighbours
# During training, negative edges are sampled on-the-fly with a ratio of 2:1.

# Define seed edges:
edge_label_index = train_data["user", "listened_to_artist", "artist"].edge_label_index
edge_label = train_data["user", "listened_to_artist", "artist"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "listened_to_artist", "artist"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "listened_to_artist", "artist"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "listened_to_artist", "artist"].edge_label.min() == 0
assert sampled_data["user", "listened_to_artist", "artist"].edge_label.max() == 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[195],
    x=[195, 47],
    n_id=[195],
    num_sampled_nodes=[3],
  },
  artist={
    node_id=[2212],
    n_id=[2212],
    num_sampled_nodes=[3],
  },
  (user, listened_to_artist, artist)={
    edge_index=[2, 15636],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[15636],
    num_sampled_edges=[2],
    input_id=[128],
  },
  (artist, rev_listened_to_artist, user)={
    edge_index=[2, 3336],
    e_id=[3336],
    num_sampled_edges=[2],
  }
)


### Creating Heterogeneous Link-level GNN


In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_artist: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_artist = x_artist[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_artist).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and artist:
        self.artist_lin = torch.nn.Linear(0, hidden_channels)
        self.user_lin = torch.nn.Linear(47, hidden_channels)
        self.artist_emb = torch.nn.Embedding(data_artist["artist"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data_artist.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_lin(data["user"].x),
          "artist": self.artist_emb(data["artist"].node_id),
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["artist"],
            data["user", "listened_to_artist", "artist"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=64)

print(model)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Model(
  (artist_lin): Linear(in_features=0, out_features=128, bias=True)
  (user_lin): Linear(in_features=47, out_features=128, bias=True)
  (artist_emb): Embedding(340364, 128)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__listened_to_artist__artist): SAGEConv(128, 128, aggr=mean)
      (artist__rev_listened_to_artist__user): SAGEConv(128, 128, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__listened_to_artist__artist): SAGEConv(128, 128, aggr=mean)
      (artist__rev_listened_to_artist__user): SAGEConv(128, 128, aggr=mean)
    )
  )
  (classifier): Classifier()
)


### Training Heterogeneous Link-level GNN

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 6):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["user", "listened_to_artist", "artist"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 639/639 [00:09<00:00, 69.10it/s]


Epoch: 001, Loss: 0.1894


100%|██████████| 639/639 [00:09<00:00, 69.49it/s]


Epoch: 002, Loss: 0.1280


...

Epoch: 003, Loss: 0.1154


100%|██████████| 639/639 [00:08<00:00, 74.59it/s]


Epoch: 004, Loss: 0.1029


100%|██████████| 639/639 [00:09<00:00, 67.27it/s]

Epoch: 005, Loss: 0.0910


### Evaluating Heterogeneous Link-level GNN


In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "listened_to_artist", "artist"].edge_label_index
edge_label = val_data["user", "listened_to_artist", "artist"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "listened_to_artist", "artist"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "listened_to_artist", "artist"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "listened_to_artist", "artist"].edge_label.min() >= 0
assert sampled_data["user", "listened_to_artist", "artist"].edge_label.max() <= 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[188],
    x=[188, 47],
    n_id=[188],
    num_sampled_nodes=[3],
  },
  artist={
    node_id=[1918],
    n_id=[1918],
    num_sampled_nodes=[3],
  },
  (user, listened_to_artist, artist)={
    edge_index=[2, 17051],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[17051],
    num_sampled_edges=[2],
    input_id=[384],
  },
  (artist, rev_listened_to_artist, user)={
    edge_index=[2, 2990],
    e_id=[2990],
    num_sampled_edges=[2],
  }
)


In [ ]:
preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data["user", "listened_to_artist", "artist"].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Calculate ROC-AUC
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

# Calculate Average Precision (AP)
ap = average_precision_score(ground_truth, pred)
print(f"Average Precision (AP): {ap:.4f}")

# Calculate F1 score
# First, we need to convert the probabilities to class labels (0 or 1)
# We use 0.5 as the threshold
pred_labels = (pred > 0.5).astype(int)
f1 = f1_score(ground_truth, pred_labels)
print(f"F1 Score: {f1:.4f}")

100%|██████████| 266/266 [00:01<00:00, 173.30it/s]



Validation AUC: 0.9869
Average Precision (AP): 0.9799
F1 Score: 0.9525
